## Using GCN on the CommonsenseQA dataset

In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd
import torch.nn.functional as F

from torch.autograd import Variable

import Graph_Conv_Network
import conceptnet
import importlib
importlib.reload(conceptnet)
importlib.reload(Graph_Conv_Network)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


<module 'Graph_Conv_Network' from '/home/sean/Documents/UCL/Statistical NLP/Commonsense/commonsenseqa-master (2)/Graph_Conv_Network.py'>

In [2]:
CN = conceptnet.ConceptNet()

In [4]:
CN.load_file()
CN.build_vocab()
CN.load_triplets()
CN.get_source_concept()


Iteration:  256  ,Danger Word:  tomales
Iteration:  1349  ,Danger Word:  janeiro
Iteration:  1729  ,Danger Word:  tomales
Iteration:  1742  ,Danger Word:  webvan
Iteration:  2417  ,Danger Word:  rica
Iteration:  2697  ,Danger Word:  desination
Iteration:  2880  ,Danger Word:  aren
Iteration:  3630  ,Danger Word:  tomales
Iteration:  4001  ,Danger Word:  tomales
Iteration:  5517  ,Danger Word:  rica
Iteration:  5664  ,Danger Word:  janeiro
Iteration:  6578  ,Danger Word:  rica
Iteration:  6664  ,Danger Word:  paticular
Iteration:  7335  ,Danger Word:  poorch
Iteration:  7800  ,Danger Word:  rica
Iteration:  8339  ,Danger Word:  desination
Iteration:  9375  ,Danger Word:  rica


In [5]:
CN.construct_subgraph()

['laziness']
['apiary']
['embarassment']
['galapagos', 'archipelago']
['palpitations']
['reproducing']
['perspiration']
['loath']
['birdhouse']
['compile']
['untrue']
['veracious']
['wakefulness']
['unobservant']
['underdog']
['gruber']
['ashtray']
['bugle']
['networked']
['sleepwalking']
['peacefulness']
['contemn']
['profane']
['compost', 'pile']
['become', 'disillusioned']
['whiskers']
['blackouts']
['winch']
['marmoset']
['petri', 'dish']
['fainting']
['procreating']
['sameness']
['savvy']
['exasperation']
['scrutinizing']
['ferret']
['windowsill']
['ardor']
['feel', 'resentful']
['digital', 'dexterity']
['roadblock']
['carpal', 'tunnel', 'syndrome']
['calculator']
['wristwatch']
['might', 'awaken']
['sleepwalking']
['windowsill']
['boardroom']
['gazelle']
['commuters']
['excrete']
['feline']
['salesperson']
['ficus']
['ecosphere']
['moistener']
['hand', 'lotion']
['storybook']
['adore']
['revere']
['meditate']
['refrigerators']
['excrete']
['meow']
['emit', 'light']
['dishwasher']

['indigestion']
['tasty']
['knowlege']
['untruthful']
['veracious']
['truthful']
['blaring']
['refreshment']
['lower', 'cholesterol']
['intoxication']
['cornet']
['laziness']
['blowfish']
['fiddling']
['syllogism']
['overeating']
['heartburn']
['indigestion']
['lemur']
['bodice']
['exaggerate']
['incompetent']
['flooring']
['conclave']
['mortuary']
['hells', 'canyon']
['south', 'lake', 'tahoe']
['martorell']
['randomness']
['aggravation']
['deceptive']
['altruistic']
['feild']
['martorell']
['hallucination']
['misfire']
['cashier']
['decaying', 'matter']
['latency']
['cathodes']
['voltaic', 'cell']
['electrolytic', 'cell']
['tourniquet']
['clutter']
['belligerent']
['bells', 'and', 'whistles']
['detest']
['jellyfish']
['dishwasher']
['obstruct', 'justice']
['trashcan']
['spell', 'checker']
['procreate']
['reproduce', 'asexually']
['artichoke']
['windowsill']
['windowsill']
['esoteric']
['arcane']
['drugstore']
['backache']
['deceive']
['septic', 'tank']
['disorientation']
['blowfish']


['marmot']
['dishwasher']
['multitask']
['companionable']
['drugstore']
['vagina']
['composted']
['salesperson']
['mistrial']
['very', 'adaptable']
['playroom']
['deli']
['drugstore']
['revere']
['adore']
['meat', 'grinder']
['playroom']
['shaker']
['needlepoint']
['livingroom']
['housework']
['artichoke']
['condiments']
['alpenstock']
['arrowhead']
['bookshelf']
['stagger']
['bbq']
['cramps']
['sprinting']
['physical', 'exertion']
['antipathy']
['snore']
['restlessness']
['suburbia']
['mutate']
['infect']
['euphoria']
['shrinkage']
['truthful']
['roadblock']
['carpal', 'tunnel', 'syndrome']
['rodent']
['chipmunk']
['insubstantial']
['inconsequential']
['indistinct']
['meditate']
['playfulness']
['compliments']
['punishing']
['empowering']
['martorell']
['headaches']
['tomales', 'bay']
['acid', 'reflux']
['acknowledgment']
['obsessive', 'compulsive']
['analyse']
['bushel', 'basket']
['comforter']
['livingroom']
['reproducing']
['overpopulation']
['hurtful']
['hangover']
['drunkenness']

['laundromat']
['accomplishing']
['sportsmanlike']
['silverware']
['stressful']
['fayetteville']
['cornet']
['competitiveness']
['saucepan']
['heifer']
['stockyard']
['indeterminate']
['fleas']
['dexterity']
['respiration']
['expiration']
['misrepresent']
['deceitful']
['dishonesty']
['ream']
['elective']
['cleverest']
['bitterest']
['embarassment']
['farmyard']
['fairgrounds']
['plentiful']
['potence']
['competency']
['binge', 'drinking']
['australian', 'outback']
['will', 'not', 'starve']
['breathlessness']
['roadblock']
['dignify']
['humiliate']
['demean']
['cartilage']
['poor', 'workmanship']
['bookends']
['judgemental']
['awaking']
['tiredness']
['automaton']
['meow']
['networked']
['blowfish']
['drowsiness']
['whorehouse']
['incompetent']
['exaggerate']
['petting', 'zoo']
['dishwasher']
['marmoset']
['burrow']
['beehive']
['make', 'patchwork', 'quilt']
['procrastinate']
['underbrush']
['windowsill']
['solidity']
['openness']
['hollowness']
['socialize']
['embarassment']
['calluse

['contagious']
['diaper', 'bag']
['janitor']
['carrots']
['bug', 'campers']
['trucker']
['diaper', 'bag']
['handshake']
['run', 'amok']
['analyse']
['typewriter']
['pedalling']
['obstructing']
['committing', 'perjury']
['stationery', 'store']
['dek']
['bubblegum']
['personal', 'cleanliness']
['bookcase']
['planetarium']
['bathtub']
['smartness']
['thrift', 'store']
['lawnmower']
['become', 'disillusioned']
['unprotected']
['stagnation']
['ignoble']
['plebeian']
['tradesmen']
['dard']
['tasty']
['indigestion']
['lumberyard']
['reproducing']
['shortness', 'of', 'breath']
['vocation']
['multivitamin']
['procreation']
['tiredness']
['forgetfulness']
['slurred', 'speech']
['aviary']
['cemetary']
['heifer']
['corral']
['do', 'crossword', 'puzzle']
['cumbersome']
['snowflake']
['teardrops']
['visualize']
['keg']
['procrastinate']
['refrigerators']
['clutter']
['petted']
['lactose']
['indigestion']
['stapler']
['digesting']
['saliva', 'nd', 'mouth']
['spittle']
['phlegm']
['blunderbuss']
['cak

In [5]:
class Flatten(nn.Module):
    def forward(self, dictionary): 
        #dictionary contains input and adj. we want input
        input = dictionary['input']
        return input.view(input.size()[0],-1)

In [6]:
def batch_handler_for_GCN(batch_size, CN):
    #returns a list of inputs/outputs in batches of batch_size
    #CN is a conceptnet instance
    n_examples = len(CN.choices)
    
    '''
    feed_list should be populated with dictionaries that have two keys, 'input' and 'adj',
    the value of 'input' is a matrix of dimension (batch_size x n_nodes x feature_size)
    the value of 'adj' is a matrix of dimension (batch_size x n_nodes x n_nodes)
    '''
    feed_list = []
    
    '''
    label_list should be populated with matrices of dimension (batch_size x 1)
    '''
    label_list = [] #should be populated with labels 1,2,3
    
    #convert A,B,C into 0,1,2
    labels = [torch.tensor(('B'==label) + ('C' == label) *2) for label in CN.labels]
    
    n_batches = int(n_examples/batch_size)
    
    for i in range(n_batches):
        start_index = batch_size*i
        end_index = batch_size*(i+1)
        
        temp_labels = torch.tensor(labels[start_index:end_index])
        temp_adj = CN.adjacency_mat[start_index][None,:]
        temp_feat = CN.Gfeature_mat[start_index][None,:]
        
        for j in range(start_index + 1, end_index):
            temp_adj = torch.cat((temp_adj, CN.adjacency_mat[j][None,:]), dim = 0)
            temp_feat = torch.cat((temp_feat, CN.Gfeature_mat[j][None,:]), dim = 0)
            
        feed_list.append({'input':Variable(temp_feat), 'adj':Variable(temp_adj)})
        label_list.append(temp_labels)
    return feed_list, label_list

In [48]:
model = nn.Sequential(Graph_Conv_Network.GCN(768, 40), 
                      Graph_Conv_Network.GCN(40, 10),
                      Flatten(), 
                      nn.Linear(40, 3))

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.01, betas=(0.7, 0.99))

In [49]:
def train(epochs, batch_size, CN):
    feed_list, label_list = batch_handler_for_GCN(batch_size, CN)
    feed_list = feed_list[0:700]
    label_list = label_list[0:700]
    for i in range(epochs):
        
        Loss = 0
        for in_dict, labels in zip(feed_list, label_list):
            # zero gradient
            model.zero_grad()
            
            pred = model(in_dict)
            loss = criterion(pred, labels)
            
            loss.backward()
            
            optimizer.step()
            
            Loss += loss
            
        print('Epoch', i, ':', Loss)
train(50, 10, CN)

Epoch 0 : tensor(776.0021, grad_fn=<AddBackward0>)
Epoch 1 : tensor(775.3018, grad_fn=<AddBackward0>)
Epoch 2 : tensor(767.9209, grad_fn=<AddBackward0>)
Epoch 3 : tensor(756.6613, grad_fn=<AddBackward0>)
Epoch 4 : tensor(743.8000, grad_fn=<AddBackward0>)
Epoch 5 : tensor(731.9654, grad_fn=<AddBackward0>)
Epoch 6 : tensor(721.8985, grad_fn=<AddBackward0>)
Epoch 7 : tensor(714.4681, grad_fn=<AddBackward0>)
Epoch 8 : tensor(704.7141, grad_fn=<AddBackward0>)
Epoch 9 : tensor(694.9174, grad_fn=<AddBackward0>)
Epoch 10 : tensor(685.8521, grad_fn=<AddBackward0>)
Epoch 11 : tensor(676.3712, grad_fn=<AddBackward0>)
Epoch 12 : tensor(669.1588, grad_fn=<AddBackward0>)
Epoch 13 : tensor(661.0582, grad_fn=<AddBackward0>)
Epoch 14 : tensor(656.3052, grad_fn=<AddBackward0>)
Epoch 15 : tensor(655.3368, grad_fn=<AddBackward0>)
Epoch 16 : tensor(650.2952, grad_fn=<AddBackward0>)
Epoch 17 : tensor(647.2018, grad_fn=<AddBackward0>)
Epoch 18 : tensor(640.6349, grad_fn=<AddBackward0>)
Epoch 19 : tensor(632.

# Sadly, GCN seems to be no more better than random....

In [50]:
def test(batch_size, CN):
    feed_list, label_list = batch_handler_for_GCN(batch_size, CN)
    feed_list =  feed_list[700:]
    label_list = label_list[700:]
    correct = 0
    total = 0
    for in_dict, labels in zip(feed_list, label_list):
        # zero gradient
        model.zero_grad()

        pred = model(in_dict)
        correct += ((labels == pred.argmax(dim = 1)).sum())
        total += 10
    print(correct.float()/total)

test(10, CN)
    

tensor(0.2808)
